In [56]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 500)
import os
import re
import matplotlib as mt
import matplotlib.pyplot as plt
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
import sklearn
from sklearn import cluster
%matplotlib inline 
#makes matplotlib show up! 

In [57]:
traffic = pd.read_csv('/Users/eric_ha/Desktop/Traffic Data/us-traffic-2015/dot_traffic_2015.txt.gz', compression='gzip',
                         header=0, sep=',', quotechar='"')

In [58]:
traffic.head(10)
#showing first 10 rows

,date,day_of_data,day_of_week,direction_of_travel,direction_of_travel_name,fips_state_code,functional_classification,functional_classification_name,lane_of_travel,month_of_data,record_type,restrictions,station_id,traffic_volume_counted_after_0000_to_0100,traffic_volume_counted_after_0100_to_0200,traffic_volume_counted_after_0200_to_0300,traffic_volume_counted_after_0300_to_0400,traffic_volume_counted_after_0400_to_0500,traffic_volume_counted_after_0500_to_0600,traffic_volume_counted_after_0600_to_0700,traffic_volume_counted_after_0700_to_0800,traffic_volume_counted_after_0800_to_0900,traffic_volume_counted_after_0900_to_1000,traffic_volume_counted_after_1000_to_1100,traffic_volume_counted_after_1100_to_1200,traffic_volume_counted_after_1200_to_1300,traffic_volume_counted_after_1300_to_1400,traffic_volume_counted_after_1400_to_1500,traffic_volume_counted_after_1500_to_1600,traffic_volume_counted_after_1600_to_1700,traffic_volume_counted_after_1700_to_1800,traffic_volume_counted_after_1800_to_1900,traffic_volume_counted_after_1900_to_2000,traffic_volume_counted_after_2000_to_2100,traffic_volume_counted_after_2100_to_2200,traffic_volume_counted_after_2200_to_2300,traffic_volume_counted_after_2300_to_2400,year_of_data
0,2015-04-07,7,3,1,North,56,3R,Rural: Principal Arterial - Other,1,4,3,NaN,000084,4,3,2,4,43,78,116,144,132,115,150,184,169,136,129,89,122,124,110,69,73,28,12,6,15
1,2015-09-26,26,7,7,West,21,1U,Urban: Principal Arterial - Interstate,2,9,3,NaN,056P94,381,252,218,194,220,348,453,679,826,962,1158,1379,1376,1383,1453,1617,1669,1308,1068,928,885,798,650,613,15
2,2015-06-16,16,3,3,East,6,1U,Urban: Principal Arterial - Interstate,0,6,3,NaN,077590,585,408,328,364,696,1929,4228,5634,5673,4636,3925,3827,4049,3954,4077,4244,4405,4609,4361,3272,2243,2050,1453,892,15
3,2015-04-26,26,1,5,South,55,1U,Urban: Principal Arterial - Interstate,1,4,3,NaN,450001,105,73,68,66,77,113,254,367,487,668,870,996,1003,1000,1043,1011,959,851,708,559,457,297,207,110,15
4,2015-05-23,23,7,3,East,4,4R,Rural: Minor Arterial,0,5,3,NaN,102210,6,4,2,3,1,10,17,52,64,68,82,96,99,87,87,83,61,55,35,29,21,23,9,7,15
5,2015-07-25,25,7,5,South,44,2U,Urban: Principal Arterial - Other Freeways or ...,0,7,3,NaN,008148,1168,781,738,425,279,395,715,1202,1486,1819,2193,2401,2691,2508,2700,2673,2746,2564,2257,2140,2060,2156,1873,1590,15
6,2015-09-10,10,5,3,East,19,3U,Urban: Principal Arterial - Other,2,9,3,NaN,000823,34,43,27,48,88,269,334,476,444,355,375,434,492,499,539,542,571,559,395,351,262,217,153,61,15
7,2015-10-27,27,3,3,East,36,4U,Urban: Minor Arterial,1,10,3,NaN,009582,13,3,8,3,17,47,95,158,151,151,141,184,193,219,217,238,278,250,154,130,84,56,21,14,15
8,2015-06-26,26,6,7,West,42,1R,Rural: Principal Arterial - Interstate,2,6,3,NaN,000393,115,78,31,40,42,134,268,335,392,441,600,722,652,706,751,740,784,822,703,425,192,159,150,168,15
9,2015-05-12,12,3,5,South,40,2U,Urban: Principal Arterial - Other Freeways or ...,2,5,3,NaN,AVC068,189,100,115,108,219,527,1212,1596,1347,1115,1114,1126,1178,1193,1364,1662,1673,1519,1148,914,808,555,468,270,15


In [59]:
traffic_clean = traffic.drop(['direction_of_travel','fips_state_code', 'lane_of_travel', 'year_of_data', 'record_type', 'restrictions', 'station_id', 'day_of_data', 'direction_of_travel_name','functional_classification'], axis=1)

In [60]:
traffic_clean.head(10)

,date,day_of_week,functional_classification_name,month_of_data,traffic_volume_counted_after_0000_to_0100,traffic_volume_counted_after_0100_to_0200,traffic_volume_counted_after_0200_to_0300,traffic_volume_counted_after_0300_to_0400,traffic_volume_counted_after_0400_to_0500,traffic_volume_counted_after_0500_to_0600,traffic_volume_counted_after_0600_to_0700,traffic_volume_counted_after_0700_to_0800,traffic_volume_counted_after_0800_to_0900,traffic_volume_counted_after_0900_to_1000,traffic_volume_counted_after_1000_to_1100,traffic_volume_counted_after_1100_to_1200,traffic_volume_counted_after_1200_to_1300,traffic_volume_counted_after_1300_to_1400,traffic_volume_counted_after_1400_to_1500,traffic_volume_counted_after_1500_to_1600,traffic_volume_counted_after_1600_to_1700,traffic_volume_counted_after_1700_to_1800,traffic_volume_counted_after_1800_to_1900,traffic_volume_counted_after_1900_to_2000,traffic_volume_counted_after_2000_to_2100,traffic_volume_counted_after_2100_to_2200,traffic_volume_counted_after_2200_to_2300,traffic_volume_counted_after_2300_to_2400
0,2015-04-07,3,Rural: Principal Arterial - Other,4,4,3,2,4,43,78,116,144,132,115,150,184,169,136,129,89,122,124,110,69,73,28,12,6
1,2015-09-26,7,Urban: Principal Arterial - Interstate,9,381,252,218,194,220,348,453,679,826,962,1158,1379,1376,1383,1453,1617,1669,1308,1068,928,885,798,650,613
2,2015-06-16,3,Urban: Principal Arterial - Interstate,6,585,408,328,364,696,1929,4228,5634,5673,4636,3925,3827,4049,3954,4077,4244,4405,4609,4361,3272,2243,2050,1453,892
3,2015-04-26,1,Urban: Principal Arterial - Interstate,4,105,73,68,66,77,113,254,367,487,668,870,996,1003,1000,1043,1011,959,851,708,559,457,297,207,110
4,2015-05-23,7,Rural: Minor Arterial,5,6,4,2,3,1,10,17,52,64,68,82,96,99,87,87,83,61,55,35,29,21,23,9,7
5,2015-07-25,7,Urban: Principal Arterial - Other Freeways or ...,7,1168,781,738,425,279,395,715,1202,1486,1819,2193,2401,2691,2508,2700,2673,2746,2564,2257,2140,2060,2156,1873,1590
6,2015-09-10,5,Urban: Principal Arterial - Other,9,34,43,27,48,88,269,334,476,444,355,375,434,492,499,539,542,571,559,395,351,262,217,153,61
7,2015-10-27,3,Urban: Minor Arterial,10,13,3,8,3,17,47,95,158,151,151,141,184,193,219,217,238,278,250,154,130,84,56,21,14
8,2015-06-26,6,Rural: Principal Arterial - Interstate,6,115,78,31,40,42,134,268,335,392,441,600,722,652,706,751,740,784,822,703,425,192,159,150,168
9,2015-05-12,3,Urban: Principal Arterial - Other Freeways or ...,5,189,100,115,108,219,527,1212,1596,1347,1115,1114,1126,1178,1193,1364,1662,1673,1519,1148,914,808,555,468,270


In [61]:
traffic_clean.rename(columns={'traffic_volume_counted_after_0000_to_0100':'Traffic_Volume_at_12:00AM-1:00AM',
                              'traffic_volume_counted_after_0100_to_0200':'Traffic_Volume_at_1:00AM-2:00AM',
                              'traffic_volume_counted_after_0200_to_0300':'Traffic_Volume_at_2:00AM-3:00AM',
                              'traffic_volume_counted_after_0300_to_0400':'Traffic_Volume_at_3:00AM-4:00AM',
                              'traffic_volume_counted_after_0400_to_0500':'Traffic_Volume_at_4:00AM-5:00AM',
                              'traffic_volume_counted_after_0500_to_0600':'Traffic_Volume_at_5:00AM-6:00AM',
                              'traffic_volume_counted_after_0600_to_0700':'Traffic_Volume_at_6:00AM-7:00AM',
                              'traffic_volume_counted_after_0700_to_0800':'Traffic_Volume_at_7:00AM-8:00AM',
                              'traffic_volume_counted_after_0800_to_0900':'Traffic_Volume_at_8:00AM-9:00AM',
                              'traffic_volume_counted_after_0900_to_1000':'Traffic_Volume_at_9:00AM-10:00AM',
                              'traffic_volume_counted_after_1000_to_1100':'Traffic_Volume_at_10:00AM-11:00AM',
                              'traffic_volume_counted_after_1100_to_1200':'Traffic_Volume_at_11:00AM-12:00PM',
                              'traffic_volume_counted_after_1200_to_1300':'Traffic_Volume_at_12:00PM-1:00PM',
                              'traffic_volume_counted_after_1300_to_1400':'Traffic_Volume_at_1:00PM-2:00PM',
                              'traffic_volume_counted_after_1400_to_1500':'Traffic_Volume_at_2:00PM-3:00PM',
                              'traffic_volume_counted_after_1500_to_1600':'Traffic_Volume_at_3:00PM-4:00PM',
                              'traffic_volume_counted_after_1600_to_1700':'Traffic_Volume_at_4:00PM-5:00PM',
                              'traffic_volume_counted_after_1700_to_1800':'Traffic_Volume_at_5:00PM-:600PM',
                              'traffic_volume_counted_after_1800_to_1900':'Traffic_Volume_at_6:00PM-7:00PM',
                              'traffic_volume_counted_after_1900_to_2000':'Traffic_Volume_at_7:00PM-8:00PM',
                              'traffic_volume_counted_after_2000_to_2100':'Traffic_Volume_at_8:00PM-9:00PM',
                              'traffic_volume_counted_after_2100_to_2200':'Traffic_Volume_at_9:00PM-10:00PM',
                              'traffic_volume_counted_after_2200_to_2300':'Traffic_Volume_at_10:00PM-11:00PM',
                              'traffic_volume_counted_after_2300_to_2400':'Traffic_Volume_at_11:00PM-12:00AM'}, inplace=True)

In [62]:
#traffic_clean = traffic_clean[~traffic_clean['functional_classification_name'].str.contains('\\Rural')]

traffic_clean = traffic_clean[traffic_clean.functional_classification_name.str.match('^[^R]')]
#clean all Rural roads


In [63]:
traffic_clean.head(10)

,date,day_of_week,functional_classification_name,month_of_data,Traffic_Volume_at_12:00AM-1:00AM,Traffic_Volume_at_1:00AM-2:00AM,Traffic_Volume_at_2:00AM-3:00AM,Traffic_Volume_at_3:00AM-4:00AM,Traffic_Volume_at_4:00AM-5:00AM,Traffic_Volume_at_5:00AM-6:00AM,Traffic_Volume_at_6:00AM-7:00AM,Traffic_Volume_at_7:00AM-8:00AM,Traffic_Volume_at_8:00AM-9:00AM,Traffic_Volume_at_9:00AM-10:00AM,Traffic_Volume_at_10:00AM-11:00AM,Traffic_Volume_at_11:00AM-12:00PM,Traffic_Volume_at_12:00PM-1:00PM,Traffic_Volume_at_1:00PM-2:00PM,Traffic_Volume_at_2:00PM-3:00PM,Traffic_Volume_at_3:00PM-4:00PM,Traffic_Volume_at_4:00PM-5:00PM,Traffic_Volume_at_5:00PM-:600PM,Traffic_Volume_at_6:00PM-7:00PM,Traffic_Volume_at_7:00PM-8:00PM,Traffic_Volume_at_8:00PM-9:00PM,Traffic_Volume_at_9:00PM-10:00PM,Traffic_Volume_at_10:00PM-11:00PM,Traffic_Volume_at_11:00PM-12:00AM
1,2015-09-26,7,Urban: Principal Arterial - Interstate,9,381,252,218,194,220,348,453,679,826,962,1158,1379,1376,1383,1453,1617,1669,1308,1068,928,885,798,650,613
2,2015-06-16,3,Urban: Principal Arterial - Interstate,6,585,408,328,364,696,1929,4228,5634,5673,4636,3925,3827,4049,3954,4077,4244,4405,4609,4361,3272,2243,2050,1453,892
3,2015-04-26,1,Urban: Principal Arterial - Interstate,4,105,73,68,66,77,113,254,367,487,668,870,996,1003,1000,1043,1011,959,851,708,559,457,297,207,110
5,2015-07-25,7,Urban: Principal Arterial - Other Freeways or ...,7,1168,781,738,425,279,395,715,1202,1486,1819,2193,2401,2691,2508,2700,2673,2746,2564,2257,2140,2060,2156,1873,1590
6,2015-09-10,5,Urban: Principal Arterial - Other,9,34,43,27,48,88,269,334,476,444,355,375,434,492,499,539,542,571,559,395,351,262,217,153,61
7,2015-10-27,3,Urban: Minor Arterial,10,13,3,8,3,17,47,95,158,151,151,141,184,193,219,217,238,278,250,154,130,84,56,21,14
9,2015-05-12,3,Urban: Principal Arterial - Other Freeways or ...,5,189,100,115,108,219,527,1212,1596,1347,1115,1114,1126,1178,1193,1364,1662,1673,1519,1148,914,808,555,468,270
12,2015-10-11,1,Urban: Principal Arterial - Other,10,162,91,74,71,50,51,110,164,250,381,532,544,620,472,484,444,419,423,433,345,311,255,308,124
14,2015-11-23,2,Urban: Principal Arterial - Other,11,23,13,20,13,45,135,429,691,713,604,581,545,540,563,507,515,483,478,324,198,97,89,79,41
15,2015-05-09,7,Urban: Principal Arterial - Interstate,5,20,21,17,26,16,16,49,94,112,104,122,108,146,166,149,192,178,167,109,106,93,100,61,33


In [ ]:
outputFile=os.path.join("/Users/eric_ha/Desktop/Traffic Data/us-traffic-2015/Traffic_Data_Cleaned_URBAN_ONLY.csv")


#change output path depending


